# Integrating Big Data Analytics and Convolutional Neural Networks for Pest and Disease Detection and Classification

In [1]:
#List the contents of the root directory in HDFS
!hdfs dfs -ls /

Found 4 items
drwxrwxr-x+  - hduser supergroup          0 2024-03-13 11:10 /Cropdiseasedata
drwxr-xr-x   - hduser supergroup          0 2024-03-07 10:06 /output1
drwxr-xr-x   - hduser supergroup          0 2024-03-07 13:14 /output2
drwxr-xr-x   - hduser supergroup          0 2024-03-14 14:03 /user1


In [2]:
#sc master - running  locally
sc.master

'local[*]'

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from pyspark.sql import SParkSession
from pyspark.ml.image import ImageSchema
#Initialize Spark session
spark = SparkSession.builder \
    .appName("imageprocessing") \
    .getOrCreate()

#Path to the directory containing the data in Hadoop
crop_data = "hdfs://9000/Cropdiseasedata/CCMT-Dataset-Augmented"


In [4]:
#Spark Context
sc = spark.sparkContext

In [ ]:
from pyspark.sql import SparkSession

#Initialize Spark session
spark = SparkSession.builder \
    .appName("imageprocessing") \
    .getOrCreate()

# Path to your image data
cropdata = "file:///home/hduser/Big data and advanced analytics data"
# Stop SparkSession
spark.stop()

In [8]:
from pyspark.sql import SparkSession

#Initialize Spark session
spark = SparkSession.builder \
    .appName("localimageprocessing") \
    .getOrCreate()

# Path to your image data
cropdata = "file:///home/hduser/Big data and advanced analytics data/"
# Stop SparkSession
spark.stop()

In [17]:
# spark is from the previous example.
sc = spark.sparkContext

# Path to your image data
cropdata = "file:///home/hduser/Big data and advanced analytics data"

# Read image files into a DataFrame
crop_dataframe = spark.read.format("image").load(cropdata)

# Show the DataFrame schema and first few rows
crop_dataframe.printSchema()
crop_dataframe.show()

# Stop SparkSession
spark.stop()

24/03/16 20:17:57 WARN SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)

+-----+
|image|
+-----+
+-----+



In [16]:
# The inferred schema can be visualized using the printSchema() method
crop_dataframe.printSchema()

AttributeError: 'DataFrameReader' object has no attribute 'printSchema'

In [ ]:
!git add .
!git commit -m "comment"
!git push origin main

[master ac4686d] comment
 3 files changed, 240 insertions(+)
 create mode 100644 .ipynb_checkpoints/Semester_Two_CA_CODES-checkpoint.ipynb
 create mode 100644 .~lock.conference-template-a4.docx#
Username for 'https://github.com': 

In [ ]:
file = sc.textFile

In [1]:
#Import all libraries required for EDA, Preprocessing, Model building, Model Testing, Model EValuation and Visualization
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from keras.models import Sequential
from tensorflow.compat.v1.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
#Default size for input images
width=256
height=256
depth=3epoch_ = 25
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
root_dir = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/'
INIT_LR = 1e-3

In [ ]:
#Convert Image into NUmPy array
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [ ]:
#Image and Lable List
image_list, label_list = [], []

In [ ]:
try:

In [2]:
import PIL

In [3]:
print("Pillow Version:", PIL.__version__)

Pillow Version: 9.0.1


In [ ]:
#Load all the images in a directory
from os import listdir
from matplotlib import image
#Load all images i